In [1]:
from keras.datasets import mnist
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape)
train_images=train_images.reshape(train_images.shape[0],train_images.shape[1]*train_images.shape[2])
print(train_images.shape)
test_images=test_images.reshape(test_images.shape[0],test_images.shape[1]*test_images.shape[2])
print(train_images.shape)

Using TensorFlow backend.


(60000, 28, 28)
(60000, 784)
(60000, 784)


In [2]:
# normalize training img and test img
train_images_normalised = train_images/255.0
test_images_normalised = test_images/255.0

In [3]:
print(np.mean(train_images_normalised))
train_images_normalised_copy=train_images_normalised.copy()
for i in range(train_images_normalised.shape[0]):
        for j in range(train_images_normalised.shape[1]):
            if (train_images_normalised[i][j] > 0.5):
                train_images_normalised_copy[i][j]=1
            else:
                train_images_normalised_copy[i][j]=0

print(train_images_normalised_copy.shape)

0.1306604762738429
(60000, 784)


In [4]:
def plot_digit(some_digit):
    
    some_digit_image = some_digit.reshape(28,28)

    plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation = "nearest")
    plt.axis("off")
    plt.show()

In [5]:

#print(train_images_normalised[0].shape)
#plot_digit(train_images_normalised[30])
#plot_digit(train_images_normalised_copy[30])


In [6]:
def find_edge(data):
    edge=[]
    sign=0
    b=0
    for i in range(0,28): 
        sign=0
        for j in range(0,28):
            if data[i][j]==0 and sign==1:
                edge.append(((i*28+j),(i*28+j-1)))
            if data[i][j]==0:
                sign=1
            else:
                sign=0
            if data[i][j]==1:
                b=b+1
    for j in range(0,28): 
        sign=0
        for i in range(0,28):
            if data[i][j]==0 and sign==1:
                edge.append(((i*28+j),((i-1)*28+j)))
            if data[i][j]==0:
                sign=1
            else:
                sign=0
    return edge,b

In [7]:
#print(train_images_normalised[30].shape)
#testlist,count_b=find_edge(train_images_normalised_copy[30].reshape(28,28))

#print(count_b)

In [35]:
"""
class QuickUnion(object):
    id=[]
    count=0
    
    def __init__(self,n):
        self.count = n
        i=0
        while i<n:
            self.id.append(i)
            i+=1

    def connected(self,p,q):
        if self.find(p) == self.find(q):
            return True
        else:            
            return False
    
    def find(self,p):   
        while (p != self.id[p]):
            p = self.id[p]
        return p
    
    def union(self,p,q):
        idq = self.find(q)
        idp = self.find(p)
        if not self.connected(p,q):
            self.id[idp]=idq
            self.count -=1
            
    def __delete__(self):
        print("deleted in descriptor object")
        #del self.count
        # del self.id
        
"""

In [9]:
res_train=np.zeros((train_images_normalised.shape[0],(train_images_normalised.shape[1]+1)))
print(res_train.shape)

(60000, 785)


In [10]:
"""
for i in range (train_images_normalised_copy.shape[0]):
    testlist,count_b=find_edge(train_images_normalised_copy[i].reshape(28,28))
    print(count_b)
    qf = QuickUnion(28*28)
    for k in testlist:
        p =  k[0]
        q =  k[1]
        qf.union(p,q)
    
    
    print("res:",qf.count-count_b)
    res_train[i]=np.append(train_images_normalised[i],qf.count-count_b)
    testlist=[]
    count_b=0
    qf=0
"""

'\nfor i in range (train_images_normalised_copy.shape[0]):\n    testlist,count_b=find_edge(train_images_normalised_copy[i].reshape(28,28))\n    print(count_b)\n    qf = QuickUnion(28*28)\n    for k in testlist:\n        p =  k[0]\n        q =  k[1]\n        qf.union(p,q)\n    \n    \n    print("res:",qf.count-count_b)\n    res_train[i]=np.append(train_images_normalised[i],qf.count-count_b)\n    testlist=[]\n    count_b=0\n    qf=0\n'

In [11]:
"""
testlist,count_b=find_edge(train_images_normalised_copy[2].reshape(28,28))
print(count_b)
qf = QuickUnion(28*28)
for k in testlist:
    p =  k[0]
    q =  k[1]
    qf.union(p,q)
    
print("qf:",qf.count)    
print("res:",qf.count-count_b)
"""

'\ntestlist,count_b=find_edge(train_images_normalised_copy[2].reshape(28,28))\nprint(count_b)\nqf = QuickUnion(28*28)\nfor k in testlist:\n    p =  k[0]\n    q =  k[1]\n    qf.union(p,q)\n    \nprint("qf:",qf.count)    \nprint("res:",qf.count-count_b)\n'

In [36]:
def test(input):
    testlist,count_b=find_edge(input.reshape(28,28))
    qf2 = QuickUnion(28*28)
    for k in testlist:
        p =  k[0]
        q =  k[1]
        qf2.union(p,q)
    print("qf:",qf2.count)    
    print("res:",qf2.count-count_b)
    #del qf2.count
    del qf2




In [37]:
#test(train_images_normalised_copy[2])


qf: 82
res: 1


In [38]:
#test(train_images_normalised_copy[2])

qf: 784
res: 703


In [ ]:
for i in range (train_images_normalised_copy.shape[0]):
    class QuickUnion(object):
        id=[]
        count=0
    
        def __init__(self,n):
            self.count = n
            i=0
            while i<n:
                self.id.append(i)
                i+=1

        def connected(self,p,q):
            if self.find(p) == self.find(q):
                return True
            else:            
                return False
    
        def find(self,p):   
            while (p != self.id[p]):
                p = self.id[p]
            return p
    
        def union(self,p,q):
            idq = self.find(q)
            idp = self.find(p)
            if not self.connected(p,q):
                self.id[idp]=idq
                self.count -=1
            

    testlist,count_b=find_edge(train_images_normalised_copy[i].reshape(28,28))
    print(count_b)
    qf = QuickUnion(28*28)
    for k in testlist:
        p =  k[0]
        q =  k[1]
        qf.union(p,q)
    
    
    print("res:",qf.count-count_b)
    res_train[i]=np.append(train_images_normalised[i],qf.count-count_b)
    testlist=[]
    count_b=0
    qf=0